In [ ]:
import pandas as pd
import numpy as np
import os
from helper import *
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]

In [ ]:
# simply put osm files into the data-dir (all will get evaluated)
EVAL_DIR = "/home/felix/todo/osm-tmp"
SRTM_DIR = "/home/felix/todo/osm/srtm"
# kmeans, merge, hop
MLP_METHODS = ["kmeans", "merge"]
MLP_LAYERS = [250, 500, 750, 1_000, 1_500, 2_000, 3_000, 4_000, 6_000, 8_000, 12_000, 16_000]
FAST_QUERY_METHODS = ["crp", "pch", "prp"]
QUERY_METHODS = ["normal", "bi"] + FAST_QUERY_METHODS
EVAL_FILE = os.path.join(EVAL_DIR, "saarland.eval")
EVAL_AMOUNTS = 10_000
AREAS = ["saarland"]

In [ ]:
df = pd.DataFrame()
for area in AREAS:
    for method in QUERY_METHODS:
        for mlp in MLP_METHODS:
            for partitions in MLP_LAYERS:
                if method == "normal" or method == "bi":
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + method + "-time.json")
                else:
                    tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + mlp + "-" + str(partitions) + "-" + method + "-time.json")
                tmp["method"] = method
                tmp["area"] = area
                tmp["mlp"] = mlp
                tmp["mlp_partitions"] = partitions
                df = df.append(tmp, ignore_index = True)

In [ ]:
data = list()
labels = list()
for area in AREAS:
    for method in FAST_QUERY_METHODS:
        for partitions in MLP_LAYERS:
            for mlp_method in MLP_METHODS:
                tmp = df[(df.method == method) & (df.area == area) & (df.mlp_partitions == partitions) & (df.mlp == mlp_method)]
                data.append(tmp["time"])
                labels.append(method + "-" + area + "-" + str(partitions))

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Evaluation')
ax.boxplot(data, labels=labels, showmeans=True)
plt.xticks(rotation=30, ha='right')
plt.savefig("comparison.png")
plt.show()

In [ ]:
markers = ['o', '+', 'x', '*', '.']
for area in AREAS:
    dijkstra = df[(df.method == "normal") & (df.area == area)]["time"].mean()
    bidijkstra = df[(df.method == "bi") & (df.area == area)]["time"].mean()
    for (i, method) in enumerate(FAST_QUERY_METHODS):
        color = plt.cm.Set1(i)
        for (mlp_method, marker) in zip(MLP_METHODS, markers):
            x = list()
            y = list()
            for partitions in MLP_LAYERS:
                tmp = df[(df.method == method) & (df.area == area) & (df.mlp_partitions == partitions) & (df.mlp == mlp_method)]
                x.append(partitions)
                y.append(tmp["time"].mean())
                speedup = dijkstra / tmp["time"].mean()
                print(method, partitions, speedup)
            plt.scatter(x, y, marker=marker, color=color, label=method + "-" + mlp_method)

plt.xlabel("MLP-Partition-Size")
plt.ylabel("Query-time")
plt.legend(loc='upper left')
plt.savefig("single-layer.png")
plt.show()